In [ ]:
# load init moudle file

$data_import = true
load "/work/pannote/panbot_note.rb"
nil

In [ ]:
code = <<~'TASKCODE'

# save to main-db
require 'active_record'
require 'faraday'
require 'ethereum.rb'
require 'faraday'
require 'parallel'
require 'resolv-replace'

connect_str = __PARAMS__CONNECT_STR__
http_endpoint = __PARAMS__HTTP_ENDPOINT__
block_numbers = __PARAMS__BEGIN__..__PARAMS__END__

ActiveRecord::Base.establish_connection(connect_str)


## load data model from github
models = ["application_record","address","cache","block","epoch","epoch_detail","event","transfer","tx","log"]
models.each do |model| 
  url = "https://raw.githubusercontent.com/adam429/panscan/main/panscan/app/models/#{model}.rb"
  body = Faraday.get(url).body
  f = open("load.rb","w")
  f.write(body)
  f.close
  load './load.rb'
end

## 数据准备


contract_name = "PancakePredictionV2"
contract_addr = "0x18B2A687610328590Bc8F2e5fEdDe3b582A49cdA".downcase
bscscan_apikey = "HIFNPQFV6MY755QMVPFIAY7F25RUTHI26Z"

client = Ethereum::HttpClient.new(http_endpoint)

api_url = "https://api.bscscan.com/api?module=contract&action=getabi&address=#{contract_addr}&apikey=#{bscscan_apikey}"
abi = JSON.parse(JSON.parse(response = Faraday.get(api_url).body)["result"])

contract = Ethereum::Contract.create(
        client: client, 
        name: contract_name, 
        address: contract_addr, 
        abi: abi)



get_abi = -> (contract_addr) {
  api_url = "https://api.bscscan.com/api?module=contract&action=getabi&address=#{contract_addr}&apikey=#{bscscan_apikey}"
  
  begin
    abi = JSON.parse(response = Faraday.get(api_url).body)["result"]

    while abi=="Max rate limit reached" do
      sleep(0.2)
      abi = JSON.parse(response = Faraday.get(api_url).body)["result"]
    end
  rescue Faraday::TimeoutError
    sleep(1)
    retry
  end
    
  return abi
}

decoder = Ethereum::Decoder.new
encoder = Ethereum::Encoder.new

Address.client = client
Address.get_abi = get_abi
Address.decoder = decoder

abi_constructor_inputs, abi_functions, abi_events = Ethereum::Abi.parse_abi(abi)

function_abi = abi_functions.map do |x|
   ["0x"+x.signature,x]
end.to_h

event_abi = abi_events.map do |x|
   ["0x"+x.signature,x]
end.to_h


def decode_params(decoder,log,name)
  if name=="Claim"
    return {
      "sender"=> "0x"+decoder.decode("address",log["topics"][1]),
      "epoch" => decoder.decode("uint256",log["topics"][2]),
      "amount" => decoder.decode("uint256",log["data"])/1e18
    }
  end
  if name=="BetBull"
    return {
      "sender"=> "0x"+decoder.decode("address",log["topics"][1]),
      "epoch" => decoder.decode("uint256",log["topics"][2]),
      "amount" => decoder.decode("uint256",log["data"])/1e18
    }
  end
  if name=="BetBear"
    return {
      "sender"=> "0x"+decoder.decode("address",log["topics"][1]),
      "epoch" => decoder.decode("uint256",log["topics"][2]),
      "amount" => decoder.decode("uint256",log["data"])/1e18
    }
  end
  if name=="LockRound"
    return {
      "epoch" => decoder.decode("uint256",log["topics"][1]),
      "round_id" => decoder.decode("uint256",log["topics"][2]),
      "price" => decoder.decode("int256",log["data"])/1e18
    }
  end
  if name=="EndRound"
    return {
      "epoch" => decoder.decode("uint256",log["topics"][1]),
    }
  end
  if name=="RewardsCalculated"
    data = decoder.decode_arguments(
          [
            Ethereum::FunctionInput.new({"type"=>"uint256","name"=>"rewardBaseCalAmount"}),
            Ethereum::FunctionInput.new({"type"=>"uint256","name"=>"rewardAmount"}),
            Ethereum::FunctionInput.new({"type"=>"uint256","name"=>"treasuryAmount"}),
          ], log["data"])
    return {
      "epoch" => decoder.decode("uint256",log["topics"][1]),
      "rewardBaseCalAmount" => data[0],
      "rewardAmount" => data[1],
      "treasuryAmount" => data[2],
    }
  end
  if name=="StartRound"
    return {"epoch" => decoder.decode("uint256",log["topics"][1]) }
  end
  return {}
end

def data_import_block(block_number,client,contract_addr,function_abi,event_abi,decoder)
     return "alredy exist" if Block.find_by_block_number(block_number)

    ## get block
    block = client.eth_get_block_by_number(block_number,true)["result"]
    block["block_number"] = block["number"].to_i(16)
    block["block_time"] = Time.at(block["timestamp"].to_i(16))
  
    ## get tx receipt
    transactions = block["transactions"].filter {|x| x["from"]==contract_addr.downcase or x["to"]==contract_addr.downcase }.map {|tx|  tx["block_number"]=block["block_number"]; tx["block_time"]=block["block_time"]; tx}

    transactions = Parallel.map(transactions,in_threads: 5) do |tx|
      tx["receipt"] = client.eth_get_transaction_receipt(tx["hash"])["result"]
      tx
    end
  
    ## process tx
    transactions.map! do |tx|
      size = tx["input"].size
      if tx["input"]=='0x' then
        method_hash = tx["input"]
        input_data =  tx["input"]
        tx["method_name"]= ""
        tx["input_data"] = ""
      else
        method_hash = tx["input"][0,10]
        input_data = "0x"+tx["input"][10,size]  
        if function_abi[method_hash] then
          tx["method_name"]= function_abi[method_hash].name
          tx["input_data"] = decoder.decode_arguments(function_abi[method_hash].inputs,input_data)
        else
          tx["method_name"]= ""
          tx["input_data"] = ""
        end
      end
      tx["amount"] = tx["value"].to_i(16)/1e18
      tx["block_number"] = tx["blockNumber"].to_i(16)
      tx["gas_price"] = tx["gasPrice"].to_i(16)/1e9
      tx["gas"] = tx["gas"].to_i(16)
      tx["nonce"] = tx["nonce"].to_i(16)

      tx["cumulative_gas_used"] = tx["receipt"]["cumulativeGasUsed"].to_i(16)
      tx["gas_used"] = tx["receipt"]["gasUsed"].to_i(16)

      tx["status"] = tx["receipt"]["status"]=="0x1" ? true : false

      tx["event"]=tx["receipt"]["logs"].map do |log|
        event = {}
        event["log"] = log
        event["abi"] = event_abi[log["topics"][0]]
        event["name"] = event["abi"].name
        event["params"] = decode_params(decoder,log,event["name"])
        event
      end

      tx
    end
  
    ## save to active_record
    ar = Block.new()
    ar.difficulty = block["difficulty"].to_i(16)
    ar.total_difficulty = block["totalDifficulty"].to_i(16)
    ar.block_number = block["block_number"]
    ar.gas_limit = block["gasLimit"].to_i(16)
    ar.gas_used = block["gasUsed"].to_i(16)
    ar.miner = block["miner"]
    ar.block_hash = block["hash"]
    ar.parent_hash = block["parentHash"]
    ar.block_time = Time.at(block["timestamp"].to_i(16))
    ar.block_size = block["size"].to_i(16)

    ar.save!

    transactions.map do |tx|
      ar = Tx.new()
      ar.block_hash = tx["blockHash"]
      ar.tx_hash = tx["hash"]
      ar.from = tx["from"]
      ar.to = tx["to"]
      ar.method_name = tx["method_name"]
      ar.input_data = JSON.dump(tx["input_data"])

      ar.tx_type = tx["type"].to_i(16)
      ar.tx_status = tx["status"]


      ar.block_time = tx["block_time"]
      ar.block_number = tx["blockNumber"].to_i(16)
      ar.gas = tx["gas"]
      ar.gas_price = tx["gasPrice"]
      ar.nonce = tx["nonce"]
      ar.tx_index = tx["transactionIndex"].to_i(16)
      ar.amount = tx["value"].to_i(16)
      ar.cumulative_gas_used = tx["cumulative_gas_used"]
      ar.gas_used = tx["gas_used"]
      ar.contract_address = tx["receipt"]["contractAddress"]

      begin
        ar.save!
      rescue ActiveRecord::RecordNotUnique 
      end
    end

    transactions.map {|x| x["event"]}.flatten.map do |event|
      ar = Event.new()
      ar.name = event["name"]
      ar.params = JSON.dump(event["params"])
      ar.event_abi = event["abi"].event_string
      ar.input_type = event["abi"].input_types
      ar.input_name = event["abi"].inputs
      ar.log_index = event["log"]["logIndex"].to_i(16)
      ar.tx_index = event["log"]["transactionIndex"].to_i(16)
      ar.block_hash = event["log"]["blockHash"]
      ar.tx_hash = event["log"]["transactionHash"]
      ar.block_number = event["log"]["blockNumber"].to_i(16)
      ar.event_key = "#{ar.block_number}-#{ar.tx_index}-#{ar.log_index}"

      begin
        ar.save!
      rescue ActiveRecord::RecordNotUnique 
      end
    end

  
end




time = Time.now()

puts "== data import transfer - start =="
Log.log "== data import transfer - start =="



Parallel.map(block_numbers.to_a,in_threads: 10) do |block_number|  
  puts "#{Time.now} blocks: #{block_number}" if block_number%10000==0
  Log.log "#{Time.now} blocks: #{block_number}" if block_number%10000==0
  data_import_block(block_number,client,contract_addr,function_abi,event_abi,decoder)
end

puts "== data import transfer - end =="
Log.log "== data import transfer - end =="
puts "time #{Time.now-time} s"
Log.log "time #{Time.now-time} s"

TASKCODE


"\n# save to main-db\nrequire 'active_record'\nrequire 'faraday'\nrequire 'ethereum.rb'\nrequire 'faraday'\nrequire 'parallel'\nrequire 'resolv-replace'\n\nconnect_str = __PARAMS__CONNECT_STR__\nhttp_endpoint = __PARAMS__HTTP_ENDPOINT__\nblock_numbers = __PARAMS__BEGIN__..__PARAMS__END__\n\nActiveRecord::Base.establish_connection(connect_str)\n\n\n## load data model from github\nmodels = [\"application_record\",\"address\",\"cache\",\"block\",\"epoch\",\"epoch_detail\",\"event\",\"transfer\",\"tx\",\"log\"]\nmodels.each do |model| \n  url = \"https://raw.githubusercontent.com/adam429/panscan/main/panscan/app/models/\#{model}.rb\"\n  body = Faraday.get(url).body\n  f = open(\"load.rb\",\"w\")\n  f.write(body)\n  f.close\n  load './load.rb'\nend\n\n## 数据准备\n\n\ncontract_name = \"PancakePredictionV2\"\ncontract_addr = \"0x18B2A687610328590Bc8F2e5fEdDe3b582A49cdA\".downcase\nbscscan_apikey = \"HIFNPQFV6MY755QMVPFIAY7F25RUTHI26Z\"\n\nclient = Ethereum::HttpClient.new(http_endpoint)\n\napi_u

In [ ]:

db_last_transfer_block = Transfer.order(:block_number).last.block_number
last_block = $client.eth_get_block_by_number("latest",false)["result"]["number"].to_i(16)

db_last_block = Block.order(:block_number).last.block_number
db_last_epoch = Epoch.order(:epoch).last.epoch
last_epoch = $contract.call.current_epoch
last_block = $client.eth_get_block_by_number("latest",false)["result"]["number"].to_i(16)

puts "last_epoch = #{last_epoch}"
puts "db_last_epoch = #{db_last_epoch}"
puts "-----------------------"
puts "last_block = #{last_block}"
puts "db_last_block = #{db_last_block}"
puts "-----------------------"
puts "last_transfer_block = #{db_last_transfer_block}"
puts "db_last_block = #{db_last_block}"
puts "-----------------------"

puts "epochs #{last_epoch-db_last_epoch} - blocks #{last_block-db_last_block} - transfer #{last_block-db_last_transfer_block}"
nil


last_epoch = 48405
db_last_epoch = 48404
-----------------------
last_block = 15597504
db_last_block = 15591862
-----------------------
last_transfer_block = 15263213
db_last_block = 15591862
-----------------------
epochs 1 - blocks 5642 - transfer 334291


In [ ]:
begin_block = db_last_block
end_block = last_block
step = 10000

loop do
  task_end_block = [(begin_block+step)/10000*10000,begin_block+step,end_block].min
  task_name = "data-import-block #{begin_block} - #{task_end_block}"
  task_code = code.gsub(/__PARAMS__BEGIN__/,begin_block.to_s)
                  .gsub(/__PARAMS__END__/,task_end_block.to_s)
                  .gsub(/__PARAMS__CONNECT_STR__/,'"'+ENV["DB_CONNECT_STR"]+'"')
                  .gsub(/__PARAMS__HTTP_ENDPOINT__/,'"'+ENV["HTTP_ENDPOINT"]+'"')
  puts task_name
  Task.create_task(task_name, task_code)

  begin_block = [(begin_block+step)/10000*10000,begin_block+step].min
  break if begin_block > end_block
end



data-import-block 15263213 - 15270000
data-import-block 15270000 - 15280000
data-import-block 15280000 - 15290000
data-import-block 15290000 - 15300000
data-import-block 15300000 - 15310000
data-import-block 15310000 - 15320000
data-import-block 15320000 - 15330000
data-import-block 15330000 - 15336470


In [ ]:
Block.where("block_number > ?",15263213).each do |x| x.delete end

[#<Block id: 2219190, block_number: 15263214, difficulty: 2, total_difficulty: 30333695, gas_limit: 79741422, gas_used: 20777957, miner: "0xce2fd7544e0b2cc94692d4a704debef7bcb61328", block_hash: "0xff5dd4f0debde7a8fbcb9e03306c616344d02e5c6146852c...", parent_hash: "0x3748405b69bd2cdfbc44149d5a8cb2c921369e5c41c548bc...", block_size: 31033, block_time: "2022-02-15 00:31:11.000000000 +0000", created_at: "2022-02-17 13:57:59.579275000 +0000", updated_at: "2022-02-17 13:57:59.579275000 +0000", epoch: nil>, #<Block id: 2219174, block_number: 15263215, difficulty: 2, total_difficulty: 30333697, gas_limit: 80000000, gas_used: 19866160, miner: "0xe2d3a739effcd3a99387d015e260eefac72ebea1", block_hash: "0xc9643540cd85b599d7bd4f6e0eca1c050517e80dfce04bc5...", parent_hash: "0xff5dd4f0debde7a8fbcb9e03306c616344d02e5c6146852c...", block_size: 37205, block_time: "2022-02-15 00:31:14.000000000 +0000", created_at: "2022-02-17 13:57:56.373189000 +0000", updated_at: "2022-02-17 13:57:56.373189000 +0000", 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>